In [60]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Store.SQLite"
#r "nuget: Jinaga.Notebooks"
#r "nuget: Jinaga.UnitTest"

using Jinaga;
using Jinaga.Extensions;
using Jinaga.Notebooks;
using Jinaga.Store.SQLite;
using Jinaga.UnitTest;
using System.Collections.Immutable;

Installed Packages Jinaga, 1.1.33 Jinaga.Notebooks, 1.1.8 Jinaga.Store.SQLite, 1.1.10 Jinaga.UnitTest, 1.1.4

# Jinaga Quick Start

Jinaga is a data management framework for mobile, web, and distributed applications.
It is particularly good for offline-first applications and progressive web apps, where data is stored locally and synchronized with a server.
We call that server a *replicator*, but we'll get to that.

## Jinaga Client

A Jinaga client manages data in your application and connects to a replicator.
Create a client using a factory method.
For example, to store local state in SQLite, use this method:

In [61]:
JinagaClient j = JinagaSQLiteClient.Create(options =>
{
    // Connect to the Jinaga replicator that you host yourself, or that we host for you.
    options.HttpEndpoint = new Uri("https://rep.jinaga.com/myreplicator");

    // Use the SQLite store to persist data locally.
    options.SQLitePath = System.IO.Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData),
        "jinaga", "myapplication.sqlite"
    );
});

Or to use Jinaga in a unit test, use an in-memory store:

In [62]:
JinagaClient j = JinagaTest.Create(options =>
{
    options.User = new User("--- TEST USER ---");
});

## Facts

The core unit of data in Jinaga is a *fact*.
A fact is an immutable record of an event, decision, or state change.
Let's start simple.
When the user logs into your application, that's a fact.

In [63]:
(User user, UserProfile profile) = await j.Login();

j.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER ---

Each fact has a type (`Jinaga.User` in this case) and a set of properties (`publicKey` for this example).

The `Jinaga.User` type is defined in the Jinaga library.
But, of course, you can define your own types.
Write a record and use the `FactType` attribute to define a type.

In [64]:
[FactType("Construction.Project")]
public record Project(User creator, Guid id);

To create an instance of this fact, call `j.Fact`.
This method is asynchronous because it will save the fact to the local store and notify the replicator to synchronize it with other clients.

In [65]:
Project projectA = await j.Fact(new Project(user, Guid.NewGuid()));

j.RenderFacts(projectA)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

## Predecessors

When we defined the `Construction.Project` type, we used `Jinaga.User` as one of its properties.
That made it a *predecessor*.
In this case, the meaning of that predecessor relationship is that the user created the project.

Once you have predecessors, you can query for facts that are related to them.
These, as you might have guessed, are called *successors*.

Let's query for all projects that a user has created.

In [66]:
// Create a couple more projects.
Project projectB = await j.Fact(new Project(user, Guid.NewGuid()));
Project projectC = await j.Fact(new Project(user, Guid.NewGuid()));

var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== 
 
 Construction.Project 
 
 id 
 
 20dce64d-a776-4da3-9... 
 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== 
 
 Construction.Project 
 
 id 
 
 e6d0d1fe-a934-4973-b... 
 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

Let's break down that specification.
Start with `Given` and the parameter type.
Then use `Match` to write an expression that matches the facts you want.

The `Successors` extension method finds all successors `OfType<T>` related to the given predecessor.
Provide a lambda that shows how the successors relate to the predecessor.

## Deletion

Facts are immutable, so you cannot really delete them.
Instead, you can create a new fact that marks its predecessor as deleted.

In [67]:
[FactType("Construction.Project.Deleted")]
public record ProjectDeleted(Project project, DateTime deletedAt);

var projectADeleted = await j.Fact(new ProjectDeleted(projectA, DateTime.UtcNow));

On its own, this fact does not do anything.

In [68]:
ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== 
 
 Construction.Project 
 
 id 
 
 20dce64d-a776-4da3-9... 
 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== 
 
 Construction.Project 
 
 id 
 
 e6d0d1fe-a934-4973-b... 
 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

But if you include it in the specification, it will filter out the predecessor from the results.

In [69]:
var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any())
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== 
 
 Construction.Project 
 
 id 
 
 20dce64d-a776-4da3-9... 
 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== 
 
 Construction.Project 
 
 id 
 
 e6d0d1fe-a934-4973-b... 
 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

Now if you are concerned about facts never being truly deleted, there is a way to *purge* unreachable facts.
We'll save that for later.

## Restore

For completion, let's talk about restoring a deleted fact.
You do this by defining yet another successor.

In [70]:
[FactType("Construction.Project.Restored")]
public record ProjectRestored(ProjectDeleted deleted);

await j.Fact(new ProjectRestored(projectADeleted));

Again, you need to add this fact to the specification for it to have an effect.

In [71]:
var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any(
            d => !d.Successors().OfType<ProjectRestored>(r => r.deleted).Any()))
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg== 
 
 Construction.Project 
 
 id 
 
 20dce64d-a776-4da3-9... 
 
 
<!-- X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 X2I5vMPw3DVAfpaeyuS6zK6PrianFsCBm34WlMy5bKLuq9/lpMFFq2qtRBSEnhlCeq5Ebw8wlIjuFG/eli1uVg==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA== 
 
 Construction.Project 
 
 id 
 
 e6d0d1fe-a934-4973-b... 
 
 
<!-- fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 fagUY+a8IKGL/QN826RUB1FeDlP/hpBwh3rauFyP7new7V8pQGlq8YoOdcTB6PZEPJ+WoF7oUde6Fwdx2eq+QA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

And it's back!

## Mutable Properties

Facts are immutable.
So you don't want to put a property like `name` in a project.
You would never be able to change it!

Instead, you can define a successor that represents changing the project name.
This is the pattern we use.

In [72]:
[FactType("Construction.Project.Name")]
public record ProjectName(Project project, string value, ProjectName[] prior);

That `prior` array let's you overwrite a prior value.
The first value will have an empty array, which means there is no prior value.

In [ ]:
ProjectName projectAName1 = await j.Fact(new ProjectName(projectA, "House on Cheyenne", []));

j.RenderFacts(projectAName1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project

To change the name, you create a new fact that has the old name as its prior value.

In [ ]:
ProjectName projectAName2 = await j.Fact(new ProjectName(projectA, "House on Rivercrest", [projectAName1]));

j.RenderFacts(projectAName2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 
 prior

To query for the current name, you look for the successor that has no prior value.

In [75]:
var namesOfProject = Given<Project>.Match(p =>
    p.Successors().OfType<ProjectName>(n => n.project)
        .Where(p => !p.Successors().OfType<ProjectName>(next => next.prior).Any())
);

ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ House on Rivercrest ]

### Concurrent Edits

The reason that we use the `prior` array is to handle concurrent edits.
If a user on a different device changes the name of the project, you will get a fork in the graph.

In [ ]:
ProjectName projectAName3 = await j.Fact(new ProjectName(projectA, "Home on Cheyenne", [projectAName1]));

j.RenderFacts(projectAName1, projectAName2, projectAName3)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 
 prior 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg== 
 
 Construction.Project.Name 
 
 value 
 
 Home on Cheyenne 
 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 
 prior

That will result in more than one successor matching the specification.

In [77]:
ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ House on Rivercrest, Home on Cheyenne ]

This is how you can recognize that there have been concurrent edits.
It also shows you the candidate values for the project name.

To merge, identify the correct value.
Then create a new fact that sets the correct value and has all candidates as its prior values.

In [ ]:
ProjectName projectAName4 = await j.Fact(new ProjectName(projectA, "Home on Rivercrest", names.ToArray()));

j.RenderFacts(projectAName1, projectAName2, projectAName3, projectAName4)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 Construction.Project 
 
 id 
 
 adfd2d0f-d04a-4718-b... 
 
<!-- rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 Il0TGrc+4IXIix6GU2uhl1qhDEIzkYZ7Obl5KXR+PGEwQvA0VJ0Iu7uzBzbrF6lZJE7MMjpt4HTlsR4bzOvmXg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 
 prior 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg== 
 
 Construction.Project.Name 
 
 value 
 
 Home on Cheyenne 
 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->rTggwBnZfRE5xsPv0AkUSo3OTVDV200nEgB9Md/NBnOV2JRirn21qdj558kLGqTAwmPxWR2OLYMCr8KZT8NiCQ== 
 
 
 project 
 
<!-- mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== -->
 
 mJ4POP7mHTyNseHe2UKAiBwuEgZjxEQkywObQT330hUlf4NZ0st1N0tPLztd0U7hiPTdT2b2yrnijSkMkAUosg==->NApNYI2piV8x0jyJB/htvXkIgs0uXyYacPyjSVXqbPQ+DLCxxkR+4E5V5Nss20lVMipxwc0Rq4DihMgZnOuHRQ== 
 
 
 prior 
 
<!-- 9JMsfDzot3SYGsQ+N6gQyD33+APy8ZSOM4H/2Mvx/SNOu7D7TZJJ2cmEb0I9n1WcYxLAQA7WnRGue6LVtECqvw== -->
 
 9JMsfDzot3SYGsQ+N6gQyD33+APy8ZSOM4H/2Mvx/SNOu7D7TZJJ2cmEb0I9n1WcYxLAQ

And now only one successor matches the specification.

In [79]:
ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ Home on Rivercrest ]

## Projections

You can combine specifications to create a *projection*.
This is how you will populate your user interface.

Start your specification as before, and then use the `Select` method to project the results through a new specification.

In [80]:
var projectsWithNamesCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any(
            d => !d.Successors().OfType<ProjectRestored>(r => r.deleted).Any()))
        .Select(p => new
        {
            ProjectId = p.id,
            Names = p.Successors().OfType<ProjectName>(n => n.project)
                .Where(n => !n.Successors().OfType<ProjectName>(next => next.prior).Any())
                .Select(n => n.value)
        })
);

This looks complicated, but it's just the two specifications you've already seen.
Using the `Select` method, we defined an anonymous type that contains the project name specification.

In [81]:
var projections = await j.Query(projectsWithNamesCreatedByUser, user);

projections

Error: System.ArgumentException: Unknown field type Guid, reading field ProjectId of Construction.Project.
   at Jinaga.Managers.Deserializer.DeserializeParameter(Emitter emitter, Projection projection, String parentPath, Type parameterType, String parameterName, Product product)
   at Jinaga.Managers.Deserializer.DeserializeCompoundProjection(Emitter emitter, CompoundProjection compoundProjection, Type type, ImmutableList`1 products, String path)
   at Jinaga.Managers.Deserializer.Deserialize(Emitter emitter, Projection projection, Type type, ImmutableList`1 products, String path)
   at Jinaga.Managers.FactManager.DeserializeProductsFromGraph(FactGraph graph, Projection projection, ImmutableList`1 products, Type type, String path, IWatchContext watchContext)
   at Jinaga.Managers.FactManager.ComputeProjections(Projection projection, ImmutableList`1 products, Type type, IWatchContext watchContext, String path, CancellationToken cancellationToken)
   at Jinaga.JinagaClient.RunSpecificat